In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import expipe
from expipe import Browser
import pandas as pd
import numpy as np
import sys
import exdir

sys.path.append('../ca2-mec')
import data_processing as dp

In [3]:
project_path = dp.project_path()
#project_path = '../../CA2prosjektmappe'
project = expipe.get_project(project_path)
actions = project.actions

In [4]:
dataframes = actions['dataframes']
sessions = pd.read_csv(dataframes.data_path('sessions'))
sessions.head()
print(len(sessions), len(np.unique(sessions.action)))

5 5


In [5]:
def transform_coordinates(x, y):
    """
    Transform tracking coordinates to align with physical coordinates
    For this project (CA2 MEC): rotate recorded coordinates 90 degrees, 
    followed by a shift to make values positive afterwards.
    """
    # rotate x,y coordinates 90 degrees using a 2D rotation matrix transform
    theta = np.radians(90)
    r = np.array(( (np.cos(theta), -np.sin(theta)),
                   (np.sin(theta),  np.cos(theta)) ))
    coords = r @ np.array([x,y])
    # shift new x-coordinates to be positive
    coords = coords - np.min(coords, axis=-1, keepdims=True)
    # rescale coordinates to max 1
    coords = coords / np.max(coords, axis=-1, keepdims=True)
    return coords

def socializing(x, y, top, bottom, box_locations="TL+BR"):
    """
    Defines social boundaries of the box wrt. cardinal coordinates.
    
    Params:
        top, sequence-like (2,): 2d-coordinates for end of social boundary for
                                 the social-box at the top of the environment box
        bottom, sequence-like (2,): 2d-coordinates for end of social boundary
                                    the social-box at the bottom of the environment box
        box_locations, string: either: "TL+BR", i.e. TopLeft+BottomRight or 
                                       "TR+BL", i.e. TopRight+BottomLeft.
    """
    if box_locations == "TL+BR":
        # check which indices rat is inside social boundaries
        TL_social_mask = np.stack([x < top[0], y > top[1]],axis=-1)
        TL_social_mask = np.sum(TL_social_mask, axis=-1) == 2
        BR_social_mask = np.stack([x > bottom[0], y < bottom[1]],axis=-1)
        BR_social_mask = np.sum(BR_social_mask, axis=-1) == 2
        non_social_mask = ~(TL_social_mask | BR_social_mask) # boolean operations
        return TL_social_mask, BR_social_mask, non_social_mask
    elif box_locations == "TR+BL":
        # check which indices rat is inside social boundaries
        coords = np.stack([x,y],axis=-1)
        TR_social_mask = np.sum(coords > top, axis=-1) == 2
        BL_social_mask = np.sum(coords < bottom, axis=-1) == 2
        non_social_mask = ~(TR_social_mask | BL_social_mask) # boolean operations
        return TR_social_mask, BL_social_mask, non_social_mask
    else:
        print("Choose either 'TL+BR' or 'TR+BL' as box_locations")
        return None

In [6]:
def process(data_loader: dp.Data, t_start=0.0, t_stop=None, *args, **kwargs):
    """Comments"""
    def process_row(row) -> None:
        action_id = row['action']
        x, y, t, speed = map(data_loader.tracking(action_id).get, ['x', 'y', 't', 'v'])

        # Choose tracking data time interval (IN SECONDS :D)
        if t_start is not None and t_stop is not None:
            mask = (t < t_stop) & (t > t_start)
            x, y, t = x[mask], y[mask], t[mask]
        
        total_recording_time = t[-1]        
        x, y = transform_coordinates(x, y)
        sm1, sm2, nsm = socializing(x=x, y=y, *args, **kwargs)
        
        samp_freq = len(t) / t[-1]
        top_right_time = np.sum(sm1) / samp_freq
        bottom_left_time = np.sum(sm2) / samp_freq
        sdi = (top_right_time - bottom_left_time) / (top_right_time + bottom_left_time)
        
        # Save values
        row['total-time-in-seconds'] = t[-1]
        row['time-spent-in-top-social-zone'] = top_right_time
        row['time-spent-in-bottom-social-zone'] = bottom_left_time
        row['social-discrimination-index'] = sdi
        print(row)
        
    return process_row

In [7]:
max_speed = 1 # m/s only used for speed score
min_speed = 0.02 # m/s only used for speed score
position_sampling_rate = 100 # for interpolation
position_low_pass_frequency = 6 # for low pass filtering of position

box_size = [1.0, 1.0]
bin_size=0.01
#smoothing = 0.05
baseline_duration = None

data_loader = dp.Data(
    position_sampling_rate=position_sampling_rate, 
    position_low_pass_frequency=position_low_pass_frequency,
    box_size=box_size, bin_size=bin_size, stim_mask=False, baseline_duration=baseline_duration,
)

top, bottom = np.array([0.43,0.57]), np.array([0.57,0.43])
box_locations = "TL+BR" # "TL+BR" or "TR+BL"
t_start,t_stop = 0.0,300

sessions.apply(process(data_loader, top=top, bottom=bottom, box_locations=box_locations, \
                       t_start=t_start,t_stop=t_stop), axis=1)

action                              144-100621-1
total-time-in-seconds                    299.991
time-spent-in-top-social-zone            19.7101
time-spent-in-bottom-social-zone           14.61
social-discrimination-index             0.148601
Name: 0, dtype: object
action                              144-100621-2
total-time-in-seconds                    299.997
time-spent-in-top-social-zone            101.659
time-spent-in-bottom-social-zone         27.0697
social-discrimination-index              0.57943
Name: 1, dtype: object
action                              144-100621-3
total-time-in-seconds                    299.994
time-spent-in-top-social-zone            63.4688
time-spent-in-bottom-social-zone         42.7592
social-discrimination-index             0.194954
Name: 2, dtype: object
action                              144-100621-4
total-time-in-seconds                    299.999
time-spent-in-top-social-zone            99.6797
time-spent-in-bottom-social-zone         46.9098


0    None
1    None
2    None
3    None
4    None
dtype: object

In [8]:
sessions.to_csv(dataframes.data_path('sessions'), index=False)
